In [12]:
import numpy as np
import codecs
import re

In [13]:
#Prepare training data
fname = './president/corpus.tache1.learn.utf8'
f=codecs.open(fname, 'r','utf-8') # pour régler le codage
txts=f.readlines()
nblignes=len(txts)

#cptM= 0
#cptC=0
labels = [] # M:0, C:1
trainData = []
for t in txts:
    label = re.sub(r"<[0-9]*:[0-9]*:(.)>.*","\\1",t)
    txt = re.sub(r"<[0-9]*:[0-9]*:.> (.*)","\\1",t)
    
    trainData.append(txt)   
    
    if label[0]=="C":
        #cptC+=1
        labels.append(1)
    else:
        #cptM +=1
        labels.append(0)
    
f.close()

In [14]:
from sklearn import model_selection

Train_X, Test_X, Train_Y, Test_Y = model_selection.train_test_split(trainData,labels,test_size=0.3,random_state=1)

In [15]:
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from nltk.corpus import stopwords
import nltk


Encoder = LabelEncoder()
Train_Y = Encoder.fit_transform(Train_Y)
Test_Y = Encoder.fit_transform(Test_Y)

#Tfidf_vect = TfidfVectorizer(max_features=5000)
list_stopwords = stopwords.words('french')
Tfidf_vect = TfidfVectorizer(stop_words=list_stopwords,ngram_range=(1,2))
Tfidf_vect.fit_transform(trainData)
Train_X_Tfidf = Tfidf_vect.transform(Train_X)
Test_X_Tfidf = Tfidf_vect.transform(Test_X)

In [17]:
#without Hyper-parameter Tuning
from sklearn.metrics import f1_score
from sklearn.svm import SVC

SVM = SVC(C=1.0, kernel='linear', degree=3, gamma='auto',class_weight={1: 6.6})
SVM.fit(Train_X_Tfidf,Train_Y)# predict the labels on validation dataset
predictions_SVM = SVM.predict(Test_X_Tfidf)# Use accuracy_score function to get the accuracy

#print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, Test_Y)*100)
print(f1_score(Test_Y,predictions_SVM))

0.9387033646865728


In [24]:
fname = './president/corpus.tache1.test.utf8'
s=codecs.open(fname, 'r','utf-8') # pour régler le codage
lignes_test=s.readlines()

nblignes = len(lignes_test)
test = []
for txt in lignes_test:
    txt = re.sub(r"<[0-9]*:[0-9]*> (.*)","\\1",txt)
    test.append(txt)
    
Train_X_Tfidf_test = Tfidf_vect.transform(test)
prediction = SVM.predict(Train_X_Tfidf_test)

#print(prediction)

[1 1 1 ... 1 1 1]


In [30]:
#Post-processing
prediction_post=[]
for i in range(1,len(prediction)-1):
    if prediction[i] != prediction[i-1] and prediction[i] != prediction[i+1]:
        prediction_post.append(prediction[i-1])
    else:
        prediction_post.append(prediction[i])


res_file = "./SVM/predictionPolitique.txt"
f=open(res_file,'w')
for p in prediction_post:
    if p==1:
        f.write(str('C')+"\n")
    else:
        f.write(str('M')+"\n")
            
f.close()


In [ ]:
#############################################################################################################################################

In [31]:
import os.path
def readAFile(path):
    text_file = open(path, "r")
    lines = text_file.read()
    return lines
    
path = "./movies/movies1000/"

alltxts_movie = [] # init vide
labs_movie = []
cpt = 0
for cl in os.listdir(path): # parcours des fichiers d'un répertoire
    for f in os.listdir(path+cl):
        txt = readAFile(path+cl+'/'+f)
        alltxts_movie.append(txt)
        labs_movie.append(cpt) # 0 pour neg et 1 pour pos
    cpt += 1
    #print(len(alltxts_movie))
#print(len(alltxts_movie))

neg
pos


In [32]:
Train_X_movie, Test_X_movie, Train_Y_movie, Test_Y_movie = model_selection.train_test_split(alltxts_movie,labs_movie,test_size=0.3,random_state=1)

In [33]:
Encoder_movie = LabelEncoder()
Train_Y_movie = Encoder_movie.fit_transform(Train_Y_movie)
Test_Y_movie = Encoder_movie.fit_transform(Test_Y_movie)

list_stopwords_english = stopwords.words('english')
Tfidf_vect_movie = TfidfVectorizer(stop_words=list_stopwords_english,ngram_range=(1,2))
Tfidf_vect_movie.fit_transform(alltxts_movie)
Train_X_Tfidf_movie = Tfidf_vect_movie.transform(Train_X_movie)
Test_X_Tfidf_movie = Tfidf_vect_movie.transform(Test_X_movie)

In [34]:
SVM_movie = SVC(C=1.0, kernel='linear', degree=3, gamma='auto',class_weight='balanced')
SVM_movie.fit(Train_X_Tfidf_movie,Train_Y_movie)# predict the labels on validation dataset
predictions_SVM_movie = SVM_movie.predict(Test_X_Tfidf_movie)# Use accuracy_score function to get the accuracy

print(f1_score(Test_Y_movie,predictions_SVM_movie))

0.8177339901477834


In [37]:
#Recuperation 
#Prediction du fichier test avec une ligne = 1 avis
f=open("./movies/testSentiment.txt",'r',encoding="utf8")
test_movies = f.readlines()
f.close()

In [38]:
Train_X_Tfidf_test_movie = Tfidf_vect_movie.transform(test_movies)
prediction_test_movie = SVM_movie.predict(Train_X_Tfidf_test_movie)

In [41]:
#pos==1=='C', neg==0=='M'
sortie = "./SVM/predictionMovies.txt"
f=open(sortie,'w')
for p in prediction_test_movie:
    if p==1:
        f.write(str('C')+"\n")
    else:
        f.write(str('M')+"\n")
f.close()